In [26]:
# Created by: Chen Da
# Created on: 20191031

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [28]:
# LR函数
def functionLR(id_data):
    z = intercept + np.dot(coefs, id_data.values[1:-1])
    prob = 1 / (1 + np.exp(-z))
    return prob

In [29]:
# 肺癌预测
def lungCancerPrediction(raw_df, file_name):
    df = raw_df.loc[:, selected_columns] 
    prob_list = list()
    for info in df['id'].values:
        prob = functionLR(df[df['id'] == info].iloc[0, :])
        prob_list.append(prob)
    df_pro = df.loc[:, ['id', 'class']]
    df_pro['prob'] = pd.Series(prob_list)
#     df_healthy = df_pro[df_pro['class'] == 0]
#     df_unhealthy = df_pro[df_pro['class'] == 1]
#     plt.figure(figsize=(12,10))
#     plt.scatter(df_healthy['id'], df_healthy['prob'], color='red')
#     plt.scatter(df_unhealthy['id'], df_unhealthy['prob'], color='skyblue')
#     plt.axhline(y=0.5, color='grey', linestyle='--', alpha=0.6)
#     plt.xticks(rotation=90)
#     plt.grid()
#     plt.show()
    df_pro['class'] = df_pro['class'].apply(lambda x : str(x))
    df_pro['id'] = df_pro['id'].apply(lambda x : str(x)+'_')
    df_pro = df_pro.sort_values(by='class')
    fig = px.scatter(df_pro, x='id', y='prob',
                     color='class', template='ggplot2', color_discrete_sequence=['green', '#EE0000'],
                     color_discrete_map={'0': 'green', '1': '#EE0000'})
    fig.add_trace(go.Scatter(x=[df_pro['id'].iloc[0, ], df_pro['id'].iloc[-1,]], y=[0.5, 0.5], 
                             mode='lines', 
                             line=dict(color='grey', dash='dash'),
                             showlegend=False))
    fig.update_layout(width=1200, height=800)
#     fig.to_image('C:/Users/pc/Desktop/output.png')
#     py.io.write_image(fig, '/Users/chenda/Desktop/%s' % file_name + '.png')
    fig.show()
    return df_pro

In [30]:
file_name = input("请输入原始数据的名称：")
data_path = 'C:/Users/pc/OneDrive/PLTTECH/Project/20191205_lung_cancer/rawdata/%s' % file_name + '.xlsx'
df = pd.read_excel(data_path)
df.iloc[:, 1:-1] = df.iloc[:, 1:-1].multiply(100)
df.head()

# 模型所用变量
selected_columns = ['id',
                    'Lymphocytes/CD3+',
                     'Lymphocytes/CD3+/CD4+/Q2: 158Gd_CD197_CCR7+ , 155Gd_CD45RA+',
                     'Lymphocytes/CD3+/CD8+/Granzyme B+',
                     'Lymphocytes/CD3+/CD8+/Q2: 158Gd_CD197_CCR7+ , 155Gd_CD45RA+',
                     'Lymphocytes/CD3-/B cells',
                     'Monocytes/CD14+CD16+',
                     'Monocytes/CD14+CD16-',
                     'Myeloid cells/CD56-CD14-/DC cells/mDC',
                     'Myeloid cells/HLA-DR-/MDSC',
                     'class']


# 模型系数
coefs = [-0.08390398717574404,
         -0.03610481980075306,
         0.0009420305861796553,
         -0.04410220170983452,
         0.014535886709334304,
         0.041009224243607194,
         -0.02724415803747987,
         0.08857334261005825,
         0.0003389571097505322
        ]

intercept = 0

plot_df = lungCancerPrediction(df, file_name)

请输入原始数据的名称：肺结节


In [31]:
# plot_df.to_excel('C:/Users/pc/Desktop/%s.xlsx'%file_name, index=False)

In [32]:
plot_df['pre_label'] = plot_df['prob'].apply(lambda x: round(x))
plot_df['class'] = plot_df['class'].apply(lambda x: int(x))

In [33]:
pre_0_right = plot_df[np.logical_and(plot_df['class']==0, plot_df['pre_label']==0)].shape[0]
pre_0_wrong = plot_df[np.logical_and(plot_df['class']==0, plot_df['pre_label']==1)].shape[0]
pre_1_right = plot_df[np.logical_and(plot_df['class']==1, plot_df['pre_label']==1)].shape[0]
pre_1_wrong = plot_df[np.logical_and(plot_df['class']==1, plot_df['pre_label']==0)].shape[0]
print('总体准确率为%s' % ((pre_0_right+pre_1_right)/plot_df.shape[0]))

总体准确率为0.75


In [34]:
pre_0_right, pre_0_wrong, pre_1_right, pre_1_wrong

(6, 2, 0, 0)

In [35]:
print('灵敏度为%s' % (pre_1_right/(pre_1_right+pre_1_wrong)))       # 正确判断病人的概率
print('特异度为%s' % (pre_0_right/(pre_0_right+pre_0_wrong)))       # 正确判断非病人的概率

ZeroDivisionError: division by zero